In [1]:
from mqt.bench import get_benchmark, TKETSettings, CompilerSettings
from typing import Tuple
import cudaq
import time
import re
import ast
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
# cudaq.set_target("equal1", emulate=True, url="http://host.docker.internal:62123")
cudaq.set_target("nvidia", option="fp64")

In [3]:
compiler_settings = CompilerSettings(tket=TKETSettings(placement="linplacement"))

algorithm = get_benchmark(
    benchmark_name="random",
    level="nativegates",
    circuit_size=5,
    compiler="tket",
    provider_name="ibm",
    compiler_settings=compiler_settings,
)

In [ ]:
def benchmark_algorithm(algorithm) -> Tuple[str, float]:
    kernel = cudaq.make_kernel()

    # Allocate the qubits
    for q_register in algorithm.q_registers:
        code = f"{q_register.name} = kernel.qalloc({q_register.size})"
        exec(code)

    # Translate tket commands to cudaq commands
    for command in algorithm.get_commands():
        if command.op.get_name() == "Barrier":
            continue
        elif command.op.get_name() == "Measure":
            assert len(command.qubits) == 1
            assert len(command.bits) == 1
            code = f"kernel.mz({command.qubits[0]}, '{command.bits[0]}')"
        else:
            op_name = command.op.get_name().lower()
            if len(command.op.params) > 0:
                op_name = op_name.split("(")[0]

            if op_name == "sx":
                code = f"kernel.rx(np.pi/2, {','.join([str(q_register) for q_register in command.qubits])})"
            elif op_name == "u1":
                code = re.sub(r"u1\(", "u3(0.0,0.0,", code)
            elif op_name == "u2":
                code = re.sub(r"u2\(", "u3(0.5,", code)
            elif op_name == "ccx":
                code = re.sub(
                    r"ccx\(([^)]+)\)",
                    lambda m: f"cx([{','.join(m.group(1).split(',')[:-1])}],{m.group(1).split(',')[-1]})",
                    code,
                )
            else:
                code = f"kernel.{op_name}({','.join([str(param) for param in command.op.params] + [str(q_register) for q_register in command.qubits])})"

        exec(code)

    # Sample the kernel
    start = time.perf_counter()
    counts = cudaq.sample(kernel)
    end = time.perf_counter()

    return counts, end - start

In [5]:
counts, sample_time = benchmark_algorithm(algorithm)
print(counts)
print(f"Time: {sample_time:.2f} s")

{ 
  __global__ : { 00000:72 00001:72 00010:30 00011:130 00100:13 00101:34 00110:29 00111:4 01000:12 01010:39 01011:8 01100:31 01101:40 01110:49 01111:7 10000:18 10001:32 10011:55 10100:62 10101:28 10110:1 10111:40 11000:28 11001:7 11010:7 11011:1 11100:15 11101:62 11110:35 11111:39 }
   meas[0] : { 0:570 1:430 }
   meas[1] : { 0:620 1:380 }
   meas[2] : { 0:526 1:474 }
   meas[3] : { 0:511 1:489 }
   meas[4] : { 0:441 1:559 }
}

Time: 0.46 s


In [6]:
def sample_result_to_dict(counts) -> dict:
    counts_str = str(counts)
    counts_str = re.sub(r"(\S+)\s?:", r'"\1":', counts_str)
    counts_str = counts_str.replace("}", "},")
    counts_str = re.sub(r"(\S+:\S+)", r"\1,", counts_str)
    counts_str = counts_str[:-2]

    counts_dict = ast.literal_eval(counts_str)

    return counts_dict

In [7]:
benchmarks = ["random"]
min_qubits, max_qubits = 2, 256

In [8]:
def run_benchmarks(benchmarks, min_qubits, max_qubits) -> pd.DataFrame:
    i = 0
    df = pd.DataFrame(
        columns=["benchmark", "qubits", "gates", "sample_time", "counts"],
    )
    for benchmark_name in benchmarks:
        for qubits in tqdm(range(min_qubits, max_qubits + 1), desc=benchmark_name):
            algorithm = get_benchmark(
                benchmark_name=benchmark_name,
                level="nativegates",
                circuit_size=qubits,
                compiler="tket",
                provider_name="ibm",
                compiler_settings=compiler_settings,
            )

            counts, sample_time = benchmark_algorithm(algorithm)
            counts_dict = sample_result_to_dict(counts)

            df.loc[i] = [
                algorithm.name,
                algorithm.n_qubits,
                algorithm.n_gates,
                sample_time,
                counts_dict,
            ]
            i += 1

    return df

In [ ]:
df = run_benchmarks(benchmarks, min_qubits, max_qubits)

random:   9%|▉         | 24/255 [13:26<6:50:10, 106.54s/it]

In [ ]:
df.head()

In [ ]:
df.to_csv("benchmark_results.csv", index=False)